# Test of LSDB critical functions

This notebook invokes several critical methods, necessary for the success of most users of LSDB within RSP. This does not rely on any pre-existing outside datasets, and only requires pip installation of the `lsdb` package and its dependencies.

Flow is as follows:
- generate two small, in-memory catalogs
- write the two catalogs to disk
- load the two catalogs from disk
- crossmatch the catalogs
- perform a basic summation operation

In [ ]:
# if not previously installed
# %pip install lsdb --quiet

In [19]:
import lsdb
import tempfile
from pathlib import Path

tmp_path = tempfile.TemporaryDirectory()
tmp_dir = Path(str(tmp_path))

In [20]:
region = lsdb.ConeSearch(ra=53.13, dec=-28.10, radius_arcsec=20)
expected_pixels = [lsdb.HealpixPixel(7, 143742)]

gen_catalog_1 = lsdb.generate_catalog(100, 5, seed=53, search_region=region)
assert gen_catalog_1.get_healpix_pixels() == expected_pixels
gen_catalog_1.to_hats(tmp_dir / "catalog_1", catalog_name="catalog_1", overwrite=True)

gen_catalog_2 = lsdb.generate_catalog(100, 5, seed=28, search_region=region)
assert gen_catalog_2.get_healpix_pixels() == expected_pixels
gen_catalog_2_computed = gen_catalog_2.compute()


In [21]:
catalog_1 = lsdb.open_catalog(tmp_dir / "catalog_1")
catalog_2 = lsdb.from_dataframe(gen_catalog_2_computed)

In [22]:
crossmatch_result = lsdb.crossmatch(catalog_1, gen_catalog_2)
assert len(crossmatch_result.compute()) > 0

In [23]:
def sum_id(df):
    import pandas as pd

    return pd.DataFrame([{"sum": df["id"].sum()}])


unrealized = catalog_1.map_partitions(sum_id)
assert unrealized.compute()["sum"][0] == 52752

unrealized = catalog_2.map_partitions(sum_id)
assert unrealized.compute()["sum"][0] == 56349

In [24]:
tmp_path.cleanup()